### Feature preparation

In [1]:
import pandas as pd
import os
from sklearn.utils import shuffle
import warnings
from sklearn.preprocessing import OneHotEncoder # libFM part
from sklearn.datasets import dump_svmlight_file # libFM part
import numpy as np

warnings.filterwarnings('ignore')

In [2]:
def df_shuffle(df):
    df = shuffle(df)
    df.reset_index(inplace = True)
    df.drop('index', inplace = True, axis = 1)
    return df

In [3]:
def libffm(ffm_p, tr_p, v_p, latent, cores, autostop = True, lreg = 0.00002, nrounds = 200, print_s = False, on_disc = False):
    
    if autostop:
        fm_cmd = r"%s -p %s -s %s -k %s -l %s -t %s -r 0.2 --auto-stop %s" % (ffm_p, v_p, cores, latent, str(lreg), 
                                                                                 str(nrounds), tr_p)
    else:
        fm_cmd = r"%s -p %s -s %s -k %s -l %s -t %s -r 0.2 %s" % (ffm_p, v_p, cores, latent, str(lreg), 
                                                                                 str(nrounds), tr_p)
        
    if on_disc:
        fm_cmd = r"%s  -p %s -s 4 -k 10 -t 200  --no-rand --on-disk --auto-stop %s" % (ffm_p, v_p, tr_p)
        print(os.popen(fm_cmd).read())
        
    else:
        if print_s:
            print(fm_cmd)
        log = os.popen(fm_cmd).read()
        print(log)
        try:
            result = log.split('\n')
            loss = float(result[len(result) - 4].split('      ')[2])
        except:
            loss = -1000
        print('Val logloss: ', loss)

In [4]:
def libffm_predict(ffm_p, model_p, test_p,  out_p):
    fm_cmd = r"%s %s %s %s" % (ffm_p, test_p, model_p, out_p)
    print(fm_cmd)
    try:
        os.system(fm_cmd)
        print('Predicted: 1')
    except Exception as e:
        print('Predicted: 0')
        print(e)

In [5]:
def convert_to_ffm(df,type,numerics,categories,features, bpath, model_type = ''):
        currentcode = len(numerics)
        catdict = {}
        catcodes = {}
        # Flagging categorical and numerical fields
        for x in numerics:
             catdict[x] = 0
        for x in categories:
             catdict[x] = 1

        nrows = df.shape[0]
        ncolumns = len(features)
        counter = 0
        with open(bpath + str(type) + "_" + model_type + "_ffm.txt", "w") as text_file:

            # Looping over rows to convert each row to libffm format
            for n, r in enumerate(range(nrows)):
                datastring = ""
                datarow = df.iloc[r].to_dict()
                datastring += str(int(datarow['click']))
                # For numerical fields, we are creating a dummy field here
                for i, x in enumerate(catdict.keys()):
                    if(catdict[x]==0):
                        datastring = datastring + " "+str(i)+":"+ str(i)+":"+ str(datarow[x])
                    else:
                 # For a new field appearing in a training example
                        if(x not in catcodes):
                            catcodes[x] = {}
                            currentcode +=1
                            catcodes[x][datarow[x]] = currentcode #encoding the feature
                 # For already encoded fields
                        elif(datarow[x] not in catcodes[x]):
                            currentcode +=1
                            catcodes[x][datarow[x]] = currentcode #encoding the feature
                        code = catcodes[x][datarow[x]]
                        datastring = datastring + " "+str(i)+":"+ str(int(code))+":1"

                datastring += '\n'
                text_file.write(datastring)
                if counter % 10**5 == 0:
                    print('Completed: ', round(counter / nrows, 2))
                counter += 1

In [6]:
def ftlr(path):  
    os.system("python %s" %path)

def get_hour(x):
    return int(str(x)[6:])

def get_date(x):
    return int(str(x)[:6])

dict_day = {'21' : 1, '22' : 2, '23' : 3, '24' : 4, '25' : 5, '26' : 6, '27' : 0, '28' : 1, '29' : 2, '30' : 3, '31' : 4}
def get_day(x):
    return dict_day[str(x)[4:6]]

In [7]:
base_path = 'D:/Downloads/avazu_feedzai/'
ftlr_path = r'D:\Downloads\avazu_feedzai\ftlr\ftlr.py'


libffm_path = r'D:\Downloads\avazu_feedzai\libffm\ffm-train.exe'
libffm_ftlr_path = r'D:\Downloads\avazu_feedzai\libffm-ftrl-master\libffm-ftrl-master\ffm-train.exe'
libffm_path_predict = r'D:\Downloads\avazu_feedzai\libffm\ffm-predict.exe'
libffm_ftlr_predict = r'D:\Downloads\avazu_feedzai\libffm-ftrl-master\libffm-ftrl-master\ffm-predict.exe'

train_libffm = r'D:\Downloads\avazu_feedzai\train_ffm.txt'
val_libffm = r'D:\Downloads\avazu_feedzai\val_ffm.txt'
test_libffm = r'D:\Downloads\avazu_feedzai\val_ffm.txt'

train_libffm_m = r'D:\Downloads\avazu_feedzai\train_mob_ffm.txt'
val_libffm_m = r'D:\Downloads\avazu_feedzai\val_mob_ffm.txt'
test_libffm_m = r'D:\Downloads\avazu_feedzai\test_mob_ffm.txt'

train_libffm_a = r'D:\Downloads\avazu_feedzai\train_app_ffm.txt'
val_libffm_a = r'D:\Downloads\avazu_feedzai\val_app_ffm.txt'
test_libffm_a = r'D:\Downloads\avazu_feedzai\test_app_ffm.txt'

out_a = r'D:\Downloads\avazu_feedzai\app_ffm.txt'
out_m = r'D:\Downloads\avazu_feedzai\mob_ffm.txt'

model_a = r'D:\Downloads\avazu_feedzai\train_app_ffm.txt.model'
model_m = r'D:\Downloads\avazu_feedzai\train_mob_ffm.txt.model'

In [8]:
def get_features(base_path, model_type, nrows):

    train = pd.read_csv(base_path + 'train.gz', nrows = nrows)
    # train['web'] = 1 - (train['site_id'] == '85f751fd')*1
    if model_type == 'app':
        train = train[train['site_id'] == '85f751fd']
    else:
        train = train[train['site_id'] != '85f751fd']
        
    print("Sample set shape: ",  train.shape[0])

    train_index = int(0.8*train.shape[0])

    train = df_shuffle(train)

    # process hour

    train['day'] = train['hour'].map(get_day)
    train['time'] = train['hour'].map(get_hour)
    train['date'] = train['hour'].map(get_date)

    # generate features

    # define user as device_id + device_model + device_ip

    train['user'] = train['device_id'] + train['device_model'] + train['device_ip']

    # for each user we calculate his characteristics per hour, per day, total

    for c in ['hour', 'date']:
        train['temp'] = train['user'] + train[c].astype(str)
        dicty = train.groupby('temp')['user'].count()
        train['user' + '_count_' + c] = train['temp'].map(dicty)
        print(c)
    train.drop('temp', axis = 1, inplace = True)

    for c in ['hour', 'date']:
        train['temp'] = train['user'] + train[c].astype(str)
        for cc in ['site_id', 'site_domain', 'site_category', 'app_id', 'app_domain',  'app_category', 'C14',
           'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21']:
            dicty = train.groupby('temp')[cc].nunique()
            train['user' + '_nunique_' + c + '_' + cc] = train['temp'].map(dicty)
            if abs(train[['user' + '_nunique_' + c + '_' + cc, 'click']].corr().values[0][1]) < 0.05:
                train.drop('user' + '_nunique_' + c + '_' + cc, axis = 1, inplace = True)
                print('drop user' + '_nunique_' + c + '_' + cc)
        print(c)
    train.drop('temp', axis = 1, inplace = True)

    train.head(5)

    # define an interaction of ad as site_id + app_id
    
    train['place_id'] = train['site_id'] + train['app_id']
    train['place_genre_id'] = train['site_id'] + train['app_id'] + train['site_category'] + train['app_category']
    train['tech_position'] = train['banner_pos'].astype(str) + train['device_conn_type'].astype(str)
    train['add_position'] = train['place_id'].astype(str) + train['banner_pos'].astype(str)
    train['union_category'] = train['site_category'] + train['app_category']

    train['ultra_C_type'] = train['C1'].astype(str) + train['C14'].astype(str) + train['C15'].astype(str) + train['C16'].astype(str) \
     + train['C17'].astype(str) + train['C18'].astype(str) + train['C19'].astype(str) + train['C20'].astype(str)+ train['C21'].astype(str)

    # for features we calculate how oftern they are met per hour, per day and in total

    train['user_date'] = train['user'].astype(str) + train['date'].astype(str)
    train['place_date'] = train['place_id'].astype(str) + train['date'].astype(str)

    counter = 0

    history_user = []
    history_place = []

    dict_user = {}
    dict_place = {}

    for row in train.itertuples():

        user = row[list(train.columns).index('user_date') + 1]
        place = row[list(train.columns).index('place_date') + 1]


        try:
            history_user.append(dict_user[user])
        except KeyError:
            history_user.append(0)
        try:
            history_place.append(dict_place[place])
        except KeyError:
            history_place.append(0)


        try:
            dict_user[user] += 1
        except KeyError:
            dict_user[user] = 0
        try:
            dict_place[place] += 1
        except KeyError:
            dict_place[place] = 0

        if counter % 10**6 == 0:
            print(counter / train.shape[0])
        counter += 1

    train['user_history'] = history_user
    train.ix[train['user_history'] > 10*3, 'user_history'] = 1000
    train['place_history'] = history_place
    train.ix[train['place_history'] > 10*3, 'place_history'] = 1000
    
    train.drop(['user_date', 'place_date'], axis = 1, inplace = True)

    # check how many unique values we have got

    for c in train.columns:
        print('Column %s contains %s unique values. It is %s' % (c, train[c].nunique(), round((train[c].nunique() / train.shape[0]) * 100, 2))  + ' %')

    avoid = ['click', 'hour', 'id', 'date']
    features = [t for t in train.columns if t not in avoid]

    # delete columns with 1 unique values
    drop_cols = []
    for c in features:
        if train[c].nunique() == 1:
            features.remove(c)

    # replace values with low frequency
    for c in features:
        dicty = train.groupby(c)['click'].count()
        train['temp'] = train[c].map(dicty)
        train.ix[train['temp'] < 10, c] = -999
    train.drop('temp', axis = 1, inplace = True)
    return features, train

# Save features

def save_ffm(train, bpath, name, features):
    convert_to_ffm(train,name,[],features,features, base_path)

In [9]:
# Features for app model

In [12]:
features, train = get_features(base_path, 'app', 5*10**5)
train_index = int(0.8*train.shape[0])
save_ffm(train.ix[:train_index,:], base_path, 'train', features)
save_ffm(train.ix[train_index:,:], base_path, 'val', features)

Sample set shape:  153042
hour
date
drop user_nunique_hour_app_id
drop user_nunique_hour_app_domain
drop user_nunique_hour_app_category
drop user_nunique_hour_C14
drop user_nunique_hour_C15
drop user_nunique_hour_C16
drop user_nunique_hour_C17
drop user_nunique_hour_C18
drop user_nunique_hour_C19
drop user_nunique_hour_C21
hour
drop user_nunique_date_app_id
drop user_nunique_date_app_domain
drop user_nunique_date_app_category
drop user_nunique_date_C14
drop user_nunique_date_C15
drop user_nunique_date_C16
drop user_nunique_date_C17
drop user_nunique_date_C18
drop user_nunique_date_C19
drop user_nunique_date_C21
date
0.0
Column id contains 153042 unique values. It is 100.0 %
Column click contains 2 unique values. It is 0.0 %
Column hour contains 4 unique values. It is 0.0 %
Column C1 contains 7 unique values. It is 0.0 %
Column banner_pos contains 4 unique values. It is 0.0 %
Column site_id contains 1 unique values. It is 0.0 %
Column site_domain contains 1 unique values. It is 0.0 %
Co

In [13]:
libffm(libffm_path,train_libffm, val_libffm, 10, 6, lreg = 0.00002)

First check if the text file has already converted to binary format (0.2 seconds)
Binary file found. Skip converting text to binary
First check if the text file has already converted to binary format (0.2 seconds)
Binary file found. Skip converting text to binary
iter   tr_logloss   va_logloss      tr_time
   1      0.44151      0.43087          3.0
   2      0.43039      0.42750          6.0
   3      0.42707      0.42507          9.0
   4      0.42408      0.42069         12.0
   5      0.42099      0.41699         15.0
   6      0.41731      0.41415         18.1
   7      0.41358      0.40865         21.1
   8      0.41024      0.40465         24.1
   9      0.40702      0.40129         27.1
  10      0.40385      0.39888         30.1
  11      0.40135      0.39533         33.1
  12      0.39872      0.39322         36.1
  13      0.39634      0.38992         39.1
  14      0.39400      0.38763         42.1
  15      0.39148      0.38493         45.1
  16      0.38887      0.38221  

0.2368

In [14]:
# Features for mob model

In [15]:
features, train = get_features(base_path, 'mob', 5*10**5)
train_index = int(0.8*train.shape[0])
save_ffm(train.ix[:train_index,:], base_path, 'train', features)
save_ffm(train.ix[train_index:,:], base_path, 'val', features)

Sample set shape:  346958
hour
date
drop user_nunique_hour_site_id
drop user_nunique_hour_site_domain
drop user_nunique_hour_C14
drop user_nunique_hour_C18
hour
drop user_nunique_date_site_id
drop user_nunique_date_site_domain
drop user_nunique_date_C14
drop user_nunique_date_C18
date
0.0
Column id contains 346958 unique values. It is 100.0 %
Column click contains 2 unique values. It is 0.0 %
Column hour contains 4 unique values. It is 0.0 %
Column C1 contains 5 unique values. It is 0.0 %
Column banner_pos contains 4 unique values. It is 0.0 %
Column site_id contains 1703 unique values. It is 0.49 %
Column site_domain contains 1586 unique values. It is 0.46 %
Column site_category contains 21 unique values. It is 0.01 %
Column app_id contains 1 unique values. It is 0.0 %
Column app_domain contains 1 unique values. It is 0.0 %
Column app_category contains 1 unique values. It is 0.0 %
Column device_id contains 8269 unique values. It is 2.38 %
Column device_ip contains 110964 unique values

In [16]:
libffm(libffm_path,train_libffm, val_libffm, 10, 6, lreg = 0.00002)

First check if the text file has already converted to binary format (0.2 seconds)
Binary file found. Skip converting text to binary
First check if the text file has already converted to binary format (0.2 seconds)
Binary file found. Skip converting text to binary
iter   tr_logloss   va_logloss      tr_time
   1      0.44144      0.43062          3.1
   2      0.43043      0.42717          6.1
   3      0.42706      0.42396          9.1
   4      0.42407      0.42148         12.1
   5      0.42097      0.41658         15.1
   6      0.41730      0.41304         18.1
   7      0.41362      0.40901         21.1
   8      0.41027      0.40492         24.1
   9      0.40702      0.40138         27.1
  10      0.40387      0.39911         30.1
  11      0.40133      0.39541         33.1
  12      0.39871      0.39404         36.1
  13      0.39636      0.38995         39.1
  14      0.39402      0.38731         42.1
  15      0.39152      0.38521         45.2
  16      0.38890      0.38227  

0.26844

In [17]:
# save features

In [18]:
sub = pd.DataFrame()
sub['features'] = features
sub.to_csv(base_path + 'features.csv')

In [19]:
# looks promising -> 0.2684 for mob and  0.2386 for app

### Training part for the whole data

In [52]:
# app model

In [39]:
def get_ffm_txt_full(train, base_path, model_type, features_list, delta = 10**7, sample_size = 1.0):

    if model_type == 'app':
        train = train[train['site_id'] == '85f751fd']
    else:
        train = train[train['site_id'] != '85f751fd']
        
        
    # process hour

    train['day'] = train['hour'].map(get_day)
    train['time'] = train['hour'].map(get_hour)
    train['date'] = train['hour'].map(get_date)

    # generate features

    # define user as device_id + device_model + device_ip

    train['user'] = train['device_id'] + train['device_model'] + train['device_ip']

    # for each user we calculate his characteristics per hour, per day, total

    for c in ['hour', 'date']:
        train['temp'] = train['user'] + train[c].astype(str)
        dicty = train.groupby('temp')['user'].count()
        train['user' + '_count_' + c] = train['temp'].map(dicty)
        print(c)
    train.drop('temp', axis = 1, inplace = True)

    for c in ['hour', 'date']:
        train['temp'] = train['user'] + train[c].astype(str)
        for cc in ['site_id', 'site_domain', 'site_category', 'app_id', 'app_domain',  'app_category', 'C14',
           'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21']:
            dicty = train.groupby('temp')[cc].nunique()
            train['user' + '_nunique_' + c + '_' + cc] = train['temp'].map(dicty)
        print(c)
    train.drop('temp', axis = 1, inplace = True)

    # define an interaction of ad as site_id + app_id

    train['place_id'] = train['site_id'] + train['app_id']
    train['place_genre_id'] = train['site_id'] + train['app_id'] + train['site_category'] + train['app_category']
    train['tech_position'] = train['banner_pos'].astype(str) + train['device_conn_type'].astype(str)
    train['add_position'] = train['place_id'].astype(str) + train['banner_pos'].astype(str)
    train['union_category'] = train['site_category'] + train['app_category']

    train['ultra_C_type'] = train['C1'].astype(str) + train['C14'].astype(str) + train['C15'].astype(str) + train['C16'].astype(str) \
     + train['C17'].astype(str) + train['C18'].astype(str) + train['C19'].astype(str) + train['C20'].astype(str)+ train['C21'].astype(str)

    

    train['user_date'] = train['user'].astype(str) + train['date'].astype(str)
    train['place_date'] = train['place_id'].astype(str) + train['date'].astype(str)

    counter = 0

    history_user = []
    history_place = []

    dict_user = {}
    dict_place = {}
    
    # for features user and place we calculate how oftern they are met per date in cummulative way

    for row in train.itertuples():

        user = row[list(train.columns).index('user_date') + 1]
        place = row[list(train.columns).index('place_date') + 1]


        try:
            history_user.append(dict_user[user])
        except KeyError:
            history_user.append(0)
        try:
            history_place.append(dict_place[place])
        except KeyError:
            history_place.append(0)


        try:
            dict_user[user] += 1
        except KeyError:
            dict_user[user] = 0
        try:
            dict_place[place] += 1
        except KeyError:
            dict_place[place] = 0

        if counter % 10**6 == 0:
            print(counter / train.shape[0])
        counter += 1

    train['user_history'] = history_user
    train.ix[train['user_history'] > 10*3, 'user_history'] = 1000
    train['place_history'] = history_place
    train.ix[train['place_history'] > 10*3, 'place_history'] = 1000
    
    train.drop(['user_date', 'place_date'], axis = 1, inplace = True)
    
    train = pd.concat([train[train['type'] == 1].sample(frac=sample_size, replace=False, random_state = 1),
                       train[train['type'] == 0]])
    
    train.reset_index(inplace = True)
    train.drop('index', inplace = True, axis = 1)
        
    train_shape = train[train['type'] == 1].shape[0]
    train.drop('type', axis = 1, inplace = True)
    
    train_index = int(0.8*train_shape)
    convert_to_ffm(train.ix[:train_index, :],'train',[],features_list,features_list, base_path, model_type = model_type)
    convert_to_ffm(train.ix[train_index:train_shape - 1, :],'val',[],features_list,features_list, base_path, model_type = model_type)
    convert_to_ffm(train.ix[train_shape:, :],'test',[],features_list,features_list, base_path, model_type = model_type)

In [ ]:
train = pd.read_csv(base_path + 'train.gz')
test = pd.read_csv(base_path + 'test.gz')
test['click'] = 0

train['type'] = 1
test['type'] = 0

cols = [t for t in train.columns]
data = pd.concat([train[cols], test[cols]])


features = list(pd.read_csv(base_path + 'features.csv')['features'].values)

del train
del test

In [ ]:
get_ffm_txt_full(data, base_path, 'app', features, sample_size = 0.4)

hour
date
hour
date
0.0
0.06129163165126827
0.12258326330253654
0.1838748949538048
0.24516652660507307
0.3064581582563413
0.3677497899076096
0.42904142155887787
0.49033305321014614
0.5516246848614144
0.6129163165126826
0.6742079481639509
0.7354995798152192
0.7967912114664875
0.8580828431177557
0.919374474769024
0.9806661064202923
Completed:  0.0
Completed:  0.02
Completed:  0.04
Completed:  0.06
Completed:  0.09
Completed:  0.11
Completed:  0.13
Completed:  0.15
Completed:  0.17
Completed:  0.19
Completed:  0.21
Completed:  0.24
Completed:  0.26
Completed:  0.28
Completed:  0.3
Completed:  0.32
Completed:  0.34
Completed:  0.36
Completed:  0.39
Completed:  0.41
Completed:  0.43
Completed:  0.45
Completed:  0.47
Completed:  0.49
Completed:  0.51
Completed:  0.54
Completed:  0.56
Completed:  0.58
Completed:  0.6
Completed:  0.62
Completed:  0.64
Completed:  0.66
Completed:  0.69
Completed:  0.71
Completed:  0.73
Completed:  0.75
Completed:  0.77
Completed:  0.79
Completed:  0.81


In [ ]:
# mob model

In [ ]:
train = pd.read_csv(base_path + 'train.gz')
test = pd.read_csv(base_path + 'test.gz')
test['click'] = 0

train['type'] = 1
test['type'] = 0

cols = [t for t in train.columns]
data = pd.concat([train[cols], test[cols]])


features = list(pd.read_csv(base_path + 'features.csv')['features'].values)

del train
del test

get_ffm_txt_full(data, base_path, 'mob', features)

In [25]:
# Training and prediction for app

In [ ]:
libffm(libffm_path,train_libffm_a, val_libffm_a, 10, 6, autostop = True, lreg = 0.00002, print_s = True)

In [37]:
libffm_predict(libffm_path_predict, model_a, train_libffm_a,  out_a)

D:\Downloads\avazu_feedzai\libffm\ffm-predict.exe D:\Downloads\avazu_feedzai\train_app_ffm.txt D:\Downloads\avazu_feedzai\train_app_ffm.txt.model D:\Downloads\avazu_feedzai\app_ffm.txt
Predicted: 1


In [ ]:
# Training and prediction for mob

In [9]:
libffm(libffm_path,train_libffm_m, val_libffm_m, 10, 6, lreg = 0.00002)

First check if the text file has already converted to binary format (0.0 seconds)
Binary file NOT found. Convert text file to binary file (273.6 seconds)
First check if the text file has already converted to binary format (0.0 seconds)
Binary file NOT found. Convert text file to binary file (69.3 seconds)
iter   tr_logloss   va_logloss      tr_time

Val logloss:  -1000


-1000

In [ ]:
libffm_predict(libffm_path_predict, model_m, test_libffm_m,  out_m)

In [ ]:
# FTLR predicitons

In [ ]:
ftlr(ftlr_path)

### Combine libFFM predictions and then ensemble with FTLR